In [1]:
import torch
import torchvision
# from torch.utils.tensorboard import SummaryWriter
import time
from torch import nn
from torch.utils.data import DataLoader
from load_data import MyData  # self-made
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm # View procedure
import os
import scipy.io
from random import random
import numpy as np
import gc
from torch.utils.tensorboard import SummaryWriter
from network_cnn_lstm_4 import MyNetwork
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
# exper_dir = "rest"
exper_dir = "conditionC"
root_dir = f"../data/eegmap_split/{exper_dir}"
test_data = []
test_label = []
# # ---- hc
# dataset = MyData(root_dir, f"test", "hc") # hc
# # find the fold file
# for person in range(len(dataset)):
#     filename = os.path.join(dataset.path, dataset.file_path[person])
#     data_map = torch.load(filename)
#     print(f"data_map size: {data_map.size()}")
#     # extract the pure name of the file
#     parts = filename.split("\\")
#     file_name = parts[-1]
#     name_without_extension = file_name.split(".")[0]
#     # label or data
#     file_last = name_without_extension.split("_")[-1]
#     if file_last == "label": # label
#         for label in data_map:
#             test_label.append(label)
#     else: # data
#         for data in data_map:
#             test_data.append(data)
#     del filename, parts, name_without_extension, file_last
#     gc.collect()
#     torch.cuda.empty_cache()   
# ---- mcs
dataset = MyData(root_dir, "test", "mcs") 
# find the fold file
for person in range(len(dataset)):
    filename = os.path.join(dataset.path, dataset.file_path[person])
    data_map = torch.load(filename)
    print(f"data_map size: {data_map.size()}")
    # extract the pure name of the file
    parts = filename.split("\\")
    file_name = parts[-1]
    name_without_extension = file_name.split(".")[0]
    # label or data
    file_last = name_without_extension.split("_")[-1]
    if file_last == "label": # label
        for label in data_map:
            test_label.append(label)
    else: # data
        for data in data_map:
            test_data.append(data)
    del filename, parts, name_without_extension, file_last
    gc.collect()
    torch.cuda.empty_cache()  
# ---- uws
dataset = MyData(root_dir, "test", "uws") # uws
# find the fold file
for person in range(len(dataset)):
    filename = os.path.join(dataset.path, dataset.file_path[person])
    data_map = torch.load(filename)
    print(f"data_map size: {data_map.size()}")
    # extract the pure name of the file
    parts = filename.split("\\")
    file_name = parts[-1]
    name_without_extension = file_name.split(".")[0]
    # label or data
    file_last = name_without_extension.split("_")[-1]
    if file_last == "label": # label
        for label in data_map:
            test_label.append(label)
    else: # data
        for data in data_map:
            test_data.append(data)
    del filename, parts, name_without_extension, file_last
    gc.collect()
    torch.cuda.empty_cache()  
test_data = torch.stack(test_data)
test_label = torch.stack(test_label)
print(test_data.size())
print(test_label.size())
test_data_size = test_label.size()

data_map size: torch.Size([135, 2400, 10, 11])
data_map size: torch.Size([135])
data_map size: torch.Size([122, 2400, 10, 11])
data_map size: torch.Size([122])
torch.Size([257, 2400, 10, 11])
torch.Size([257])


In [3]:
BATCH_SIZE = 1
C,H,W = 1,1,2400
learn_rate = 0.0005
num_epochs = 100
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
# test dataset
test_td = TensorDataset(test_data, test_label)
test_loader = DataLoader(test_td, batch_size = BATCH_SIZE, shuffle = True)
del test_data
del test_label
del test_td
gc.collect()
torch.cuda.empty_cache()

In [4]:
import torch.optim as optim
device = torch.device("cuda:0")
gc.collect()
torch.cuda.empty_cache()
# 加载预训练模型!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# 定义LSTM超参数
input_size = 64  # 输入特征维度
hidden_size = 64  # 隐藏单元数量
num_layers = 2  # LSTM层数
output_size = 2  # 输出类别数量

# ==损失函数权重
# ======== 二分类-HC/DOC
# 计算总样本数量
# condition1
# total_samples = 887 + 985 + 879
# condition2
# total_samples = 929 + 1029 + 886
# condition3
# total_samples = 887 + 975 + 879
# 计算每个类别的权重
# condition1
# weights = [total_samples / 887, total_samples / (985 + 879)]
# condition2
# weights = [total_samples / 929, total_samples / (1029 + 886)]
# condition3
# weights = [total_samples / 887, total_samples / (975 + 879)]
# ======== 二分类-MCS/UWS
# 计算总样本数量
# condition1
# total_samples =985 + 879
# condition2
# total_samples = 1029 + 886
# condition3
# total_samples = 975 + 879
# 计算每个类别的权重
# condition1
# weights = [total_samples / 985, total_samples / 879]
# condition2
# weights = [total_samples / 1029, total_samples / 886]
# condition3
# weights = [total_samples / 975, total_samples / 879]
# # ======== 三分类
# # 计算总样本数量
# # conditionB
# total_samples = 929 + 1029 + 886
# # 计算每个类别的权重
# # conditionB
# weights = [total_samples / 929, total_samples / 1029, total_samples / 886]

# condition3-HC/DOC
# total_samples = 887 + 975 + 879
# weights = [total_samples / 887, total_samples / (975 + 879)]
# condition3-MCS/UWS
total_samples = 975 + 879
weights = [total_samples / 975, total_samples / 879]
# 将权重转换为张量
weights_tensor = torch.tensor(weights, device=device)
# 定义交叉熵损失函数并设置权重
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
criterion = criterion.to(device)
# 将权重转换为张量
weights_tensor = torch.tensor(weights, device=device)
# 定义交叉熵损失函数并设置权重
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
criterion = criterion.to(device)
from network_cnn_lstm_4 import MyNetwork
fold = 1
begin = 3
end = 80
exper_dir = "conditionC"
classification = "mcs_uws"
model_name = "_CNN_spa_lstm"
print(f"../model/{classification}/{exper_dir}{model_name}/Fold{fold}.pt")
for i in tqdm(range(begin,end)):
    model = MyNetwork(input_size, hidden_size, num_layers, output_size)
    model.load_state_dict(torch.load(f"../model/{classification}/{exper_dir}{model_name}/Fold{fold}_Epoch{i}.pt"))
    model = model.to(device)
    # 测试步骤开始
    model.eval()
    # 初始化损失和准确率
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    total_test_loss = 0
    # 计数
    test_count = 0
    # 用于绘制混淆矩阵
    predicted_labels = torch.tensor([])
    predicted_labels = predicted_labels.to(device)
    true_labels = torch.tensor([])
    true_labels = true_labels.to(device)
    with torch.no_grad(): # 设置不进行后向传播
        for data in test_loader:
            test_count = test_count + 1
            data_map, label=data # x,y
            # x
            data_map_reshaped = torch.reshape(data_map, (110, 1, 1, 2400))
            data_map_reshaped = data_map_reshaped.to(device)
            # y
            label_int = label.long()
            label_int = label_int.to(device)
            # clear gpu
            del data_map
            del label
            gc.collect()
            torch.cuda.empty_cache()
            # y_pred
            label_pred_test = model(data_map_reshaped)

            # confusion matrix
            predicted_labels = torch.cat((predicted_labels, label_pred_test.argmax(1)), dim=0)
            true_labels = torch.cat((true_labels, label_int), dim=0)

            # loss
            loss = criterion(label_pred_test,label_int)
            total_test_loss = total_test_loss + loss.item()

            pred_label = ((label_pred_test.argmax(1)) == label_int).sum()
            # calculation
            if label_int == 1: # DOC
                if pred_label == 1: # 预测对了
                    TP = TP + 1
                elif pred_label ==0: # 预测错了
                    FN = FN + 1
            if label_int == 0: # HC
                if pred_label == 1: # 预测对了
                    TN = TN + 1
                elif pred_label == 0: # 预测错了
                    FP = FP + 1
    print(f"=====================Epoch {i} == Loss:{total_test_loss}===============")
    print(f"--Accuracy: {(TP + TN) / (TP + TN + FP + FN)}")
    print(f"--Precision: {TP / (TP + FP)}")
    print(f"--Recall: {TP / (TP + FN)}")
    del model,TP,TN,FP,FN,total_test_loss,test_count
    gc.collect()
    torch.cuda.empty_cache()

../model/mcs_uws/conditionC_CNN_spa_lstm/Fold1.pt


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:75: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/77 [00:00<?, ?it/s]

=====================Epoch 3 == Loss:100.10593042813707===============
--Accuracy: 0.8015564202334631
--Precision: 0.808695652173913
--Recall: 0.7622950819672131
=====================Epoch 4 == Loss:79.61288068778968===============
--Accuracy: 0.8910505836575876
--Precision: 0.821917808219178
--Recall: 0.9836065573770492
=====================Epoch 5 == Loss:96.65383107449765===============
--Accuracy: 0.8482490272373541
--Precision: 0.9662921348314607
--Recall: 0.7049180327868853
=====================Epoch 6 == Loss:29.640537146056246===============
--Accuracy: 0.953307392996109
--Precision: 0.9741379310344828
--Recall: 0.9262295081967213
=====================Epoch 7 == Loss:107.32997982983159===============
--Accuracy: 0.8482490272373541
--Precision: 0.816793893129771
--Recall: 0.8770491803278688
=====================Epoch 8 == Loss:32.82107461770293===============
--Accuracy: 0.9494163424124513
--Precision: 1.0
--Recall: 0.8934426229508197
=====================Epoch 9 == Loss:63.2083

FileNotFoundError: [Errno 2] No such file or directory: '../model/mcs_uws/conditionC_CNN_spa_lstm/Fold1_Epoch28.pt'